<a href="https://colab.research.google.com/github/jadeacevedo/N-Gram-LM/blob/main/N_gram_Language_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# N-Gram Language Models research

language modeling is defimes as the task of predicting the next word in a sequence given the previous words. In this project , i will focus on the related problem of predicting the next character in a sequence given the previous characters.

The  goals of this project is  to:

- Understand how to compute language model probabilities using maximum likelihood estimation.
- Implement basic smoothing, back-off and interpolation.
- Have fun using a language model to probabilistically generate texts!
- Use a set of language models to perform text classification.

In [ ]:
import math, random
from collections import defaultdict

In [ ]:
################################################################################
# Utility Functions
################################################################################

COUNTRY_CODES = ['af', 'cn', 'de', 'fi', 'fr', 'in', 'ir', 'pk', 'za']

def start_pad(n):
    ''' Returns a padding string of length n to append to the front of text
        as a pre-processing step to building n-grams '''
    return '~' * n

def create_ngram_model(model_class, path, n=2, k=0):
    ''' Creates and returns a new n-gram model trained on the city names
        found in the path file '''
    model = model_class(n, k)
    with open(path, encoding='utf-8', errors='ignore') as f:
        model.update(f.read())
    return model

def create_ngram_model_lines(model_class, path, n=2, k=0):
    ''' Creates and returns a new n-gram model trained on the city names
        found in the path file '''
    model = model_class(n, k)
    with open(path, encoding='utf-8', errors='ignore') as f:
        for line in f:
            model.update(line.strip())
    return model

## Part 1: Generating N-Grams

In [ ]:
def ngrams(n, text):
    ''' Returns the ngrams of the text as tuples where the first element is
        the length-n context and the second is the character '''

    padded_text = start_pad(n) + text
    ngram_list = []
    for i in range(n, len(padded_text)):
        context = padded_text[i-n:i]
        char = padded_text[i]
        ngram_list.append((context, char))
    return ngram_list

print(ngrams(1, 'abc'))
print(ngrams(2, 'abc'))
print(ngrams(0, 'abc'))
print(ngrams(3, 'abc'))

[('~', 'a'), ('a', 'b'), ('b', 'c')]
[('~~', 'a'), ('~a', 'b'), ('ab', 'c')]
[('', 'a'), ('', 'b'), ('', 'c')]
[('~~~', 'a'), ('~~a', 'b'), ('~ab', 'c')]


## Part 2: Creating an N-gram Model


In [ ]:
################################################################################
# Basic N-Gram Model
################################################################################

class NgramModel(object):
    ''' A basic n-gram model using add-k smoothing '''

    def __init__(self, n, k):
        self.n = n
        self.k = k
        self.vocab = set()
        self.context_counts = defaultdict(lambda:0)
        self.sequence_counts = defaultdict(lambda:0)

    def get_vocab(self):
        ''' Returns the set of characters in the vocab '''

        return self.vocab

    def update(self, text):
        ''' Updates the model n-grams based on text '''

        for context, char in ngrams(self.n, text):
            self.vocab.add(char)
            self.context_counts[context] += 1
            self.sequence_counts[(context, char)] += 1


    def prob(self, context, char):
        ''' Returns the probability of char appearing after context (without smoothing) '''

        if context not in self.context_counts or self.context_counts[context] == 0:
            return 1.0 / len(self.vocab) if len(self.vocab) > 0 else 0.0
        else:
            return self.sequence_counts[(context, char)] / self.context_counts[context]

In [ ]:
### Testing code here ###
m = NgramModel(1, 0)
m.update('abab')

print("after update |abab| : ", m.get_vocab())
m.update('abcd')
print("after update:|abcd| ", m.get_vocab())
print("P('a'|'b'):", m.prob('b', 'a'))

after update |abab| :  {'a', 'b'}
after update:|abcd|  {'a', 'c', 'd', 'b'}
P('a'|'b'): 0.5


I have provided a method `random_char(self, context)` which returns a random character according to the probability distribution determined by the given context. Specifically, let $V = \{w_1, w_2, \ldots, w_n\}$ be the vocab, sorted according to Python's natural lexicographic ordering, and let $0\leq r \leq 1$ be a random number between $0$ and $1$. The method returns a character $w_i$ such that
$$\sum_{j=1}^{i-1}P(w_j|\text{context}) \leq r < \sum_{j=1}^iP(w_j|\text{context})$$

In [ ]:
def random_char(self, context):
    ''' Returns a random character based on the given context and the
        n-grams learned by this model '''
    r = random.random()
    pre_sum = 0
    for i, char in enumerate(sorted(self.vocab)):
        post_sum = pre_sum + self.prob(context, char)
        if pre_sum <= r < post_sum:
            return char
        pre_sum = post_sum

NgramModel.random_char = random_char

In [ ]:
m = NgramModel(0, 0)
m.update('abab')
m.update('abcd')
random.seed(1)
[m.random_char('') for i in range(25)]

['a',
 'c',
 'c',
 'a',
 'b',
 'b',
 'b',
 'c',
 'a',
 'a',
 'c',
 'b',
 'c',
 'a',
 'b',
 'b',
 'a',
 'd',
 'd',
 'a',
 'a',
 'b',
 'd',
 'b',
 'a']

In [ ]:
def random_text(self, length):
    ''' Returns text of the specified character length based on the
        n-grams learned by this model '''
    ### TODO ###
    result = []
    current_context = start_pad(self.n)

    for _ in range(length):
      if self.n == 0:
        char = self.random_char('')
      else :
        char = self.random_char(current_context)
      result.append(char)

      if self.n > 0:
        current_context = current_context[1:] + char

    return ''.join(result)

NgramModel.random_text = random_text

In [ ]:
m= NgramModel(1, 0)
m.update('assignment')
m.update('computational')
m.update('linguistics')
m.update('is')
random.seed(1)
m.random_text(25)

'atigngutalintastaticontig'

### Writing Shakespeare

 training your language model!  
 First let's grab some text:

In [ ]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-10-16 03:49:48--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.06s   

2025-10-16 03:49:49 (17.9 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [ ]:
m = create_ngram_model(NgramModel, 'input.txt', 2)
m.random_text(250)

"Fir, ad le the you, suere our nall blove my he any hattles my ledo\norsend ifin mince: han-lover th I\nmadebach for lambe viern Gaught,\nbut sione guill win!\nTalt?\nOur he re and groad, thathress,' th: appy heretten I witang niefter:\nTo racrague che why,"

In [ ]:
m = create_ngram_model(NgramModel, 'input.txt', 3)
m.random_text(250)

"First Creter wrone, limble Watchese here hear I am I fors! may ture my threethis parder at men;\nBut lience their\nand speak mers oddes, unspirity: if des righs.\nWhichmannot worse have of good keeping ere.\nGremove's sequity mine o' there recaused you s"

In [ ]:
m = create_ngram_model(NgramModel, 'input.txt', 4)
m.random_text(250)

"First Citizens:\nThe justice: farm:\nShe lord:\nLord, and armour not repent durst; and compassage break'st\nThat our protes of mind;\nNot put\nTo her to take in thank than thou not at hear is their shrewd.\n\nQUEEN ELIZABETH:\nI given in here's is times and h"

In [ ]:
m = create_ngram_model(NgramModel, 'input.txt', 7)
m.random_text(250)

"First Citizen:\nAy, worthy man: make trial, marshal's true; I hear this ring.\nHow say you\nhad not upon recover all they kiss to chide not; for your knowledge,\nI never standards, set down--\nAs best beware; thou shalt to this discharged me a graver step"

#### Observations



*   I could see a distinct trend pertainng to n, the number of characters in the context, has a significant influence.
Using a low n (such as 2), the model was quite bad. It created gibberish, simply random letters strung together without any meaning or organisation
But the findings became quite more amazing when I raised n to 4 or even 7. The model began producing real words, correct punctuation, and even phrases resembling those from a Shakespeare play. The text started to match the style of the training data and flow naturally. This demonstrated that the model produced far better predictions with more background information, therefore the created text seemed more realistic and less random.


---



## Part 3: Perplexity, Smoothing, and Interpolation (6 pts)

### Perplexity
How do we know whether our language model is good? we use an intrinsic evaluation method called **perplexity**.

In this part i will implement the `perplexity(self, text)` method.  A few things to keep in mind:
- Numeric underflow will most likely be a problem, so consider using logs
- Perplexity is undefined if the language model assigns any zero probabilities to the test set. In that case your code should return positive infinity - `float('inf')`.

In [ ]:
def perplexity(self, text):
    ''' Returns the perplexity of text based on the n-grams learned by
        this model '''
    log_prob_sum = 0
    if len(text) == 0:
        return float('inf')

    for context, char in ngrams(self.n, text):
        p = self.prob(context, char)
        if p == 0:
            return float('inf')
        log_prob_sum += math.log(p)

    avg_log_prob = log_prob_sum / len(text)
    perplexity = math.exp(-avg_log_prob)
    return perplexity

NgramModel.perplexity = perplexity

In [ ]:
m = NgramModel(1, 0)
m.update('abab')
m.update('abcd')
print(m.perplexity('abcd'))
print(m.perplexity('abca'))
print(m.perplexity('abcda'))

1.189207115002721
inf
1.515716566510398


### Discuss the perplexity for text that is similar and different from Shakespeare's plays.  



In [ ]:
# download and unzip the data
!wget https://computational-linguistics-class.org/homework/ngram-lms/test_data.zip
!unzip test_data.zip

--2025-10-16 03:50:31--  https://computational-linguistics-class.org/homework/ngram-lms/test_data.zip
Resolving computational-linguistics-class.org (computational-linguistics-class.org)... 185.199.110.153
Connecting to computational-linguistics-class.org (computational-linguistics-class.org)|185.199.110.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6998 (6.8K) [application/x-zip-compressed]
Saving to: ‘test_data.zip’

test_data.zip       100%[===================>]   6.83K  --.-KB/s    in 0s      

2025-10-16 03:50:31 (72.0 MB/s) - ‘test_data.zip’ saved [6998/6998]

Archive:  test_data.zip
  inflating: nytimes_article.txt     
  inflating: shakespeare_sonnets.txt  


In [ ]:
### investigate the perplexity for the two different texts ###
shakespeare_model = create_ngram_model(NgramModel, 'input.txt', n=4, k=1)

with open('nytimes_article.txt', encoding='utf-8', errors='ignore') as f:
    nytimes_text = f.read()

with open('shakespeare_sonnets.txt', encoding='utf-8', errors='ignore') as f:
    sonnets_text = f.read()
perplexity_nytimes = shakespeare_model.perplexity(nytimes_text)
perplexity_sonnets = shakespeare_model.perplexity(sonnets_text)


print(f"Perplexity of New York Times article: {perplexity_nytimes}")
print(f"Perplexity of Shakespeare Sonnets: {perplexity_sonnets}")

Perplexity of New York Times article: inf
Perplexity of Shakespeare Sonnets: inf


### Smoothing
Laplace Smoothing is described in section 3.5.1 of the [textbook](https://web.stanford.edu/~jurafsky/slp3/).  It adds 1 to each count, and since there are $V$ characters in the vocabulary and each one was incremented, we adjust the denominator to take this into account as well.

$$P_{\text{Laplace}}(w_i) = \frac{Count(w_i) + 1}{N + |V|}$$

A variant of Laplace smoothing is called Add-k smoothing or Add-epsilon smoothing. This is described in section 3.5.2 of the textbook.

In [ ]:
def prob(self, context, char):
    ''' Returns the probability of char appearing after context with add-k smoothing '''
    ### TODO ###
    context_count = self.context_counts[context]
    sequence_count = self.sequence_counts[(context, char)]
    vocab_size = len(self.vocab)

    if vocab_size == 0:
        return 0.0

    denominator = context_count + self.k * vocab_size
    if denominator == 0:
        return 0.0

    return (sequence_count + self.k) / denominator

NgramModel.prob = prob

In [ ]:
m = NgramModel(1, 1)
m.update('abab')
m.update('abcd')
m.prob('a', 'a')
m.prob('a', 'b')
m.prob('c', 'd')
m.prob('d', 'a')

print(m.prob('a', 'a'))
print(m.prob('a', 'b'))
print(m.prob('c', 'd'))
print(m.prob('d', 'a'))

0.14285714285714285
0.5714285714285714
0.4
0.25


### Interpolation

The idea of interpolation is to calculate the higher order n-gram probabilities also combining the probabilities for lower-order n-gram models. Like smoothing, this helps us avoid the problem of zeros if we haven't observed the longer sequence in our training data.


In [ ]:
################################################################################
# N-Gram Model with Interpolation
################################################################################

class NgramModelWithInterpolation(NgramModel):
    ''' An n-gram model with interpolation '''

    def __init__(self, n, k):

        super().__init__(n, k)
        self.models = {}
        self.lambdas = [1.0 / (n + 1)] * (n + 1)

    def get_vocab(self):

        vocab = set()
        for model in self.models.values():
            vocab.update(model.get_vocab())
        return vocab

    def update(self, text):

        for i in range(self.n + 1):
            if i not in self.models:
                self.models[i] = NgramModel(i, self.k)
            self.models[i].update(text)
        self.vocab = self.get_vocab()

    def prob(self, context, char):

        interpolated_prob = 0.0

        for i in range(self.n + 1):
            current_context = context[-i:] if i > 0 else ""
            interpolated_prob += self.lambdas[i] * self.models[i].prob(current_context, char)
        return interpolated_prob

    def set_lambdas(self, lambdas):
        if len(lambdas) != self.n + 1:
            raise ValueError(f"Number of lambdas must be {self.n + 1}")
        if not math.isclose(sum(lambdas), 1.0):
             raise ValueError("Lambdas must sum to 1.0")
        self.lambdas = lambdas

In [ ]:
m = NgramModelWithInterpolation(1, 0)
m.update('abab')
m.prob('a', 'a')

m.prob('a', 'b')


m = NgramModelWithInterpolation(2, 1)
m.update('abab')
m.update('abcd')
m.prob('~a', 'b')

m.prob('ba', 'b')

m.prob('~c', 'd')

m.prob('bc', 'd')
print(m.prob('a', 'a'))
print(m.prob('a', 'b'))
print(m.prob('~a', 'b'))
print(m.prob('ba', 'b'))
print(m.prob('~c', 'd'))
print(m.prob('bc', 'd'))

0.24206349206349204
0.3849206349206349
0.46825396825396826
0.43492063492063493
0.2722222222222222
0.3222222222222222
